In [6]:
! ls ../data/interim/

enwiki-meta-parquet


In [2]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("../data/interim/enwiki-meta-parquet")

In [3]:
df.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- rev_id: integer (nullable = true)
 |-- article_title: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- username: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- image: string (nullable = true)
 |-- main: string (nullable = true)
 |-- talk: string (nullable = true)
 |-- user: string (nullable = true)
 |-- user_talk: string (nullable = true)
 |-- other: string (nullable = true)
 |-- external: string (nullable = true)
 |-- template: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- minor: boolean (nullable = true)
 |-- textdata: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)



In [5]:
df.selectExpr("length(image)", "length(main)", "length(comment)").describe().show()

+-------+------------------+-----------------+-----------------+
|summary|     length(image)|     length(main)|  length(comment)|
+-------+------------------+-----------------+-----------------+
|  count|          59054350|        114536655|         84355130|
|   mean|105.13340932547729|1623.158221645289|41.20036592913792|
| stddev| 633.4221015582104|3104.617789664033|40.99137021623147|
|    min|                 1|                0|                1|
|    max|           1888067|          2313950|              255|
+-------+------------------+-----------------+-----------------+



In [5]:
to_drop = ["image", "main", "talk", "user", "user_talk", "other", "external", "template", "comment"]

In [13]:
df.selectExpr(map(lambda x: "sum(length({0}))/1000000 as {0}".format(x), to_drop)).show()

+-----------+-------------+----------+--------+---------+------------+------------+------------+-----------+
|      image|         main|      talk|    user|user_talk|       other|    external|    template|    comment|
+-----------+-------------+----------+--------+---------+------------+------------+------------+-----------+
|6208.585151|185911.113243|188.906808|8.645842| 0.486033|12750.286814|59798.492854|10895.123496|3475.462224|
+-----------+-------------+----------+--------+---------+------------+------------+------------+-----------+



In [7]:
df.drop(*to_drop).orderBy("timestamp").coalesce(10).write.parquet("../data/processed/enwiki-meta-compact", mode="overwrite")

In [9]:
df = spark.read.parquet("../data/processed/enwiki-meta-compact")
df.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- rev_id: integer (nullable = true)
 |-- article_title: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- username: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- minor: boolean (nullable = true)
 |-- textdata: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)

